In [1]:
import pandas as pd
from datasets import load_dataset

In [2]:
MODEL = "bert-base-uncased"
BATCH_SIZE = 4

In [3]:
# columns=["label", "text", "author", "subreddit", "score", "ups", "downs", "date", "created_utc", "parent_comment"]
# train_data_df = pd.read_csv("train-balanced.csv", delimiter="\t", names=columns, header=None)
# test_data_df = pd.read_csv("test-balanced.csv", delimiter="\t", names=columns, header=None)

# train_data_df.drop(columns=["author", "subreddit", "score", "ups", "downs", "date", "created_utc", "parent_comment"],axis=1, inplace=True)
# test_data_df.drop(columns=["author", "subreddit", "score", "ups", "downs", "date", "created_utc", "parent_comment"],axis=1, inplace=True)

# train_data_df = train_data_df.dropna()
# test_data_df = test_data_df.dropna()

# train_data_df.to_csv("train.csv", sep='\t')
# test_data_df.to_csv("test.csv", sep='\t')

In [4]:
# sarc_train = load_dataset('csv', data_files=['train-balanced.csv'], delimiter=",", split="train",on_bad_lines='skip')
# print(sarc_train[0])
# columns=["label", "text", "author", "subreddit", "score", "ups", "downs", "date", "created_utc", "parent_comment"]

columns=["label", "text"]
data_files = {"train": "train.csv", "test": "test.csv"}
dataset = load_dataset('csv',data_files=data_files, delimiter="\t",column_names=columns, )

Using custom data configuration default-322defad502acf09
Reusing dataset csv (C:\Users\fahad\.cache\huggingface\datasets\csv\default-322defad502acf09\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# dataset = dataset.remove_columns(["author", "subreddit", "score", "ups", "downs", "date", "created_utc", "parent_comment"])

In [6]:
dataset["train"][35431]["text"]

'But blacking out is fun'

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [8]:
def preprocess_function(examples):
    try:
        return tokenizer(examples["text"], truncation=True,padding=True)
    except TypeError as t:
        for idx, item in enumerate(examples['text']):
            if not isinstance(item, str):
                print(idx - 1, examples['text'][idx - 1])
                print(idx, item)
                print(idx + 1, examples['text'][idx + 1])
            
        #print(*examples["text"])
        raise t

In [9]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/1126 [00:00<?, ?ba/s]

  0%|          | 0/284 [00:00<?, ?ba/s]

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
from transformers import TrainingArguments, Trainer
from os import path

output_dir_path = path.join("results")
training_args = TrainingArguments(
    output_dir = output_dir_path,
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\fahad\AppData\Roaming\Python\Python37\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1125671
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1407090


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.